In [38]:
import pandas
import os

fp = os.path.join(os.path.abspath(''), '2020-01-01_2020-04-30.stats.xls')
frame = pandas.read_excel(fp)


In [39]:
"""
Group crime rate by zip code to get the count of crimes per zip code.
"""
# Remove dashes from zip-codes
def clean_zip(z):
    return str(z).split('-')[0]

frame['zip_code'] = frame['ZIP Code'].apply(clean_zip)
grouped = frame.groupby('zip_code').sum()
grouped = pandas.DataFrame(grouped[['Offense\nCount']])
grouped = grouped.rename(columns={'Offense\nCount': 'offense_count'})
grouped.dropna(inplace=True)
grouped


,offense_count
zip_code,
75082,1
77002,1245
77003,578
77004,1550
77005,304
...,...
77701,1
78414,2
78418,1


In [40]:
"""
Download boost mobile locations for each zipcode.
"""
import requests
from collections import defaultdict

frame = None
stores_in_zip = defaultdict(int)
for z in grouped.index.values:
    # print(z)
    num_stores = 0
    page = 1
    while True:
        url = f'https://boostmobile.nearestoutlet.com/cgi-bin/jsonsearch-cs.pl?showCaseInd=false&brandId=bst&results=50&zipcode={z}&page={page}'
        response = requests.get(url)
        # with open(os.path.join(os.path.abspath(''), 'outlet.json'), 'wb') as f:
        #     f.write(response.content)
        data = response.json()
        stores = data['nearestOutletResponse']['nearestlocationinfolist']['nearestLocationInfo']
        pages = data['nearestOutletResponse']['resultsFoundNum'] / 50 + 1
        page += 1
        num_stores += len(stores)
        if page > pages:
            break
    stores_in_zip[z] = num_stores


In [41]:
"""
Converted the dict declared above into a data frame.
"""
siz_frame = pandas.DataFrame([dict(zip_code=k, amount_stores=v) for k, v in stores_in_zip.items()])
siz_frame.set_index('zip_code', inplace=True)
siz_frame.to_csv(os.path.join(os.path.abspath(''), 'stores_in_zip.csv'))
siz_frame


,amount_stores
zip_code,
75082,318
77002,384
77003,380
77004,371
77005,375
...,...
77701,15
78414,18
78418,20


In [42]:
"""
Inner join the crime rate by zip code with the number of boost
mobile stores by zip code.
"""

joined = grouped.join(siz_frame, how='inner')
# Drop that weird nan index.
joined.drop(index='nan')

# Get the correlation.
joined.corr()


,offense_count,amount_stores
offense_count,1.000000,0.604612
amount_stores,0.604612,1.000000
